In [ ]:
import pandas as pd
import altair as alt
from natsort import natsorted

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [ ]:
original_data = '../Data/20250508_BARD1scores_update_FILTERED.xlsx'
agg_correlation = '/Users/ivan/Desktop/test_excel_outputs/20250718_AggMedianRCorrelation.xlsx'

In [ ]:
def get_rescore(data_file, corr_file):
    
    df = pd.read_excel(data_file)
    print('Full Dataset Length: ', len(df))
    rescore_df = df.loc[df['snvlib_lib2'].notna()]
    rescore_df = rescore_df[['target', 'pos_id', 'snvlib_lib1', 'D05_R1_lib1', 'D05_R2_lib1', 'D05_R3_lib1', 'D13_R1_lib1', 'D13_R2_lib1' ,'D13_R3_lib1',
                          'snvlib_lib2', 'D05_R1_lib2', 'D05_R2_lib2', 'D05_R3_lib2', 'D13_R1_lib2', 'D13_R2_lib2' ,'D13_R3_lib2']]
    
    no_rescore = df.loc[~(df['snvlib_lib2'].notna())]
    r_df = pd.read_excel(corr_file)
    r_df = r_df.rename(columns = {'Targets': 'target'})

    r_df['target'] = 'BARD1_X' + r_df['target']
    r_df['exon'] = r_df['exon'].astype(str)
    r_df['exon'] = 'BARD1_X' + r_df['exon']

    r_dict = dict(zip(r_df['target'], r_df['r_correlation']))

    targets_sorted = natsorted(r_df['target'].tolist())

    print('Rescore after get_rescore: ', len(rescore_df))
    print('No rescore after get_rescore: ', len(no_rescore))

    return rescore_df, r_dict, targets_sorted, no_rescore, df

In [ ]:
def prep_rescore(rescore_df, r_dict, targets_sorted):

    grouped = rescore_df.groupby('target')

    ready_to_score = []
    
    pre_dict = {}
    post_dict = {}
    
    for target, df in grouped:
        pre_dict[target] = len(df)
        lib1_r = r_dict[target]
        
        lib2_index = targets_sorted.index(target) + 1
        lib2 = targets_sorted[lib2_index]
        lib2_r = r_dict[lib2]

        
        if lib1_r > lib2_r:
            df = df.rename(columns = {'D05_R1_lib1' : 'D05_R1',
                                        'D05_R2_lib1' : 'D05_R2',
                                        'D05_R3_lib1' : 'D05_R3', 
                                        'D13_R1_lib1' : 'D13_R1',
                                        'D13_R2_lib1' : 'D13_R2',
                                        'D13_R3_lib1' : 'D13_R3',
                                        'snvlib_lib1' : 'lib_count'
                                     }
                          )
            df = df.drop(columns = ['snvlib_lib2', 'D05_R1_lib2', 'D05_R2_lib2', 'D05_R3_lib2', 'D13_R1_lib2', 'D13_R2_lib2' ,'D13_R3_lib2'])
            
            ready_to_score.append(df)

            post_dict[target] = len(df)
        elif lib2_r >= lib1_r:
            df = df.rename(columns = {'D05_R1_lib2' : 'D05_R1',
                                        'D05_R2_lib2' : 'D05_R2',
                                        'D05_R3_lib2' : 'D05_R3', 
                                        'D13_R1_lib2' : 'D13_R1',
                                        'D13_R2_lib2' : 'D13_R2',
                                        'D13_R3_lib2' : 'D13_R3',
                                        'snvlib_lib2' : 'lib_count'
                                       }
                          )
            df = df.drop(columns = ['snvlib_lib1', 'D05_R1_lib1', 'D05_R2_lib1', 'D05_R3_lib1', 'D13_R1_lib1', 'D13_R2_lib1' ,'D13_R3_lib1'])


            ready_to_score.append(df)
            post_dict[target] = len(df)
            
    to_rescore = pd.concat(ready_to_score)
    to_rescore = to_rescore.reset_index(drop = True)

    '''
    display('Pre-Process Dictionary: ', pre_dict,
          'Post-Process Dictionary: ', post_dict
         )
    '''
    return to_rescore

In [ ]:
def score(df, all_df):

    all_df = all_df[["pos", 'ref', 'exon', "allele", "pos_id", 'amino_acid_change', 'simplified_consequence', 'target']]

    value_cols = [col for col in df.columns if "D13"  in col or "D05"  in col or "lib_count" in col]
    dfmelt = pd.melt(df, id_vars=['pos_id'], value_vars=value_cols)
    dfpivot = pd.pivot(dfmelt, index='variable', columns=['pos_id'], values='value')
    dfpivot = dfpivot.rename_axis(None, axis=1)
    dfpivot = dfpivot.rename_axis(None, axis=0)


    metanames = ['D05_R1', 'D05_R2', 'D05_R3', 'D13_R1', 'D13_R2', 'D13_R3', 'lib_count']
    metadays = [5, 5, 5, 13, 13, 13, 0]

    metadf = pd.DataFrame(
        {'sample_name': metanames,
        'time': metadays,
        }
    )
    metadf = metadf.set_index('sample_name').rename_axis(None, axis=0)

    inference = DefaultInference(n_cpus=1)
    dds = DeseqDataSet(
        counts=dfpivot,
        metadata=metadf,
        design="~time",
        refit_cooks=True
    )
    
    dds.deseq2()
    contrast = ["time", 1, 0]
    stat_res = DeseqStats(dds, contrast=contrast, inference=inference)
    stat_res.summary()
    resdf = stat_res.results_df

    resdf = resdf.reset_index(names = 'pos_id')
    resdf = resdf.merge(all_df[["pos", 'ref', 'exon', "allele", "pos_id", 'amino_acid_change', 'simplified_consequence', 'target']])

    resdf = resdf[['pos_id', 'log2FoldChange', 'lfcSE', 'pos', 'allele', 'amino_acid_change', 'simplified_consequence', 'target', 'ref', 'exon']]
    to_return = resdf.rename(columns = {'log2FoldChange': 'score', 'lfcSE': 'standard_error'})
    to_return = to_return.loc[:,['pos', 'ref', 'allele', 'exon', 'target', 'simplified_consequence', 'score', 'standard_error', 'amino_acid_change']]
    
    return to_return

In [ ]:
def remake_data(rescored, no_rescore):

    no_rescore = no_rescore[["pos", 'ref', 'exon', "allele", "pos_id", 'amino_acid_change', 'simplified_consequence', 'target']]
    final_df = pd.concat([rescored, no_rescore])
    
    print('# Variants After Rescore: ', len(final_df))

    return final_df

In [ ]:
def main():
    to_rescore, r_dict, targets_sorted, no_rescore_df, all_data = get_rescore(original_data, agg_correlation)
    rescore_df = prep_rescore(to_rescore, r_dict, targets_sorted)
    rescored_df = score(rescore_df, all_data)
    final_df = remake_data(rescored_df, no_rescore_df)

In [ ]:
main()